In [8]:
from data.data import COVID19India

In [9]:
cin = COVID19India()

In [10]:
df = cin.moh_data(save=False)

In [24]:
df.tail()

,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Death
19,Ladakh,13,0,0,0
20,Uttar Pradesh,32,1,9,0
21,Uttarakhand,3,0,0,0
22,West Bengal,7,0,0,1
23,Total number of confirmed cases in India,451,41,37,9


df

In [12]:
df2 = cin.jhu_india_data()

In [13]:
df2.tail()

,date,confirmed,recovered,deaths
57,2020-03-19,194.0,15.0,4.0
58,2020-03-20,244.0,20.0,5.0
59,2020-03-21,330.0,23.0,4.0
60,2020-03-22,396.0,27.0,7.0
61,2020-03-23,396.0,27.0,7.0


In [21]:
df2.iloc[61:, 1:] = [400, 35, 9]

In [22]:
df2

,date,confirmed,recovered,deaths
0,2020-01-22,0.0,0.0,0.0
1,2020-01-23,0.0,0.0,0.0
2,2020-01-24,0.0,0.0,0.0
3,2020-01-25,0.0,0.0,0.0
4,2020-01-26,0.0,0.0,0.0
...,...,...,...,...
57,2020-03-19,194.0,15.0,4.0
58,2020-03-20,244.0,20.0,5.0
59,2020-03-21,330.0,23.0,4.0
60,2020-03-22,396.0,27.0,7.0


In [2]:
from pathlib import Path

In [3]:
import pandas as pd
import numpy as np

markdown_file = open('help.md')
file_content = markdown_file.read()
md = markdown.Markdown(extensions=['tables', 'codehilite', 'fenced_code'])
pg = md.convert(file_content)

In [4]:
url = "https://www.worldometers.info/coronavirus/"

In [7]:
dfs= pd.read_html(url)

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop
0,China,81171,+78,3277.0,7.0,73159.0,4735,1573.0,56.00
1,Italy,63927,NaN,6077.0,NaN,7432.0,50418,3204.0,1057.00
2,USA,46145,"+2,411",582.0,29.0,295.0,45268,1040.0,139.00
3,Spain,35136,NaN,2311.0,NaN,3355.0,29470,2355.0,751.00
4,Germany,29056,NaN,123.0,NaN,453.0,28480,23.0,347.00
...,...,...,...,...,...,...,...,...,...
192,Somalia,1,NaN,NaN,NaN,NaN,1,NaN,0.06
193,Syria,1,NaN,NaN,NaN,NaN,1,NaN,0.06
194,Timor-Leste,1,NaN,NaN,NaN,NaN,1,NaN,0.80
195,Turks and Caicos,1,NaN,NaN,NaN,NaN,1,NaN,26.00
